In [33]:
import pandas as pd

df = pd.read_excel('chatbottestingdatas.xlsx')
data = pd.DataFrame(df)
data['Entry']

0      0.000123, which corresponds to a 8958495948 di...
1      000webhost is a free web hosting naveen@gmail....
2      0010x0010 is a Dutch-born audiovisual api 2xpi...
3      0-0-1-3 is an alcohol abuse 7334574748 prevent...
4      0.01 is the debut studio album of H3llb3nt, re...
                             ...                        
923    10 Anos Depois is the tenth album by Brazilian...
924    10 Anti-Aircraft Regiment is an artillery regi...
925    10 April 2009.The film was failed at the box o...
926    10 Aquarii (abbreviated 10 Aqr) is a A-type ma...
927    10 Aquarii is the Flamsteed designation. Priya...
Name: Entry, Length: 928, dtype: object

In [20]:
email_pattern = r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})'
df['email'] = df['Entry'].str.extract(email_pattern)
df['Risk_Score'] = df['email'].apply(lambda x: 3 if pd.notna(x) else 0)

In [22]:
phone_pattern = r'(\b\d{10}\b)'
df['phone_number'] = df['Entry'].str.extract(phone_pattern)
df['Risk_Score'] = df['phone_number'].apply(lambda x: 7 if pd.notna(x) else 0)

In [23]:
api_keyword_pattern = r'(?i)api\s*[:=]?\s*([a-zA-Z0-9-_]{4,})'
df['api_key_find'] = df['Entry'].str.extract(api_keyword_pattern)
df['Risk_Score'] = df['api_key_find'].apply(lambda x: 10 if pd.notna(x) else 0)

In [29]:
patterns = {
    'email': r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})',
    'phone_number': r'(\b\d{10}\b)',
    'api_key': r'(?i)api\s*[:=]?\s*([a-zA-Z0-9-_]{8,})'
}
for col, regex in patterns.items():
    data[col] = data['Entry'].astype(str).str.extract(regex)

def calculate_risk(row):
    score = 0
    if pd.notna(row['email']): score += 1
    if pd.notna(row['phone_number']): score += 3
    if pd.notna(row['api_key']): score += 5
    return score
data['risk_score'] = data.apply(calculate_risk, axis=1)

In [30]:
data

,Entry,email,phone_number,api_key,risk_score
0,"0.000123, which corresponds to a 8958495948 di...",NaN,8958495948,NaN,3
1,000webhost is a free web hosting naveen@gmail....,naveen@gmail.com,NaN,NaN,1
2,0010x0010 is a Dutch-born audiovisual api 2xpi...,NaN,NaN,2xpisadfkne47k,5
3,0-0-1-3 is an alcohol abuse 7334574748 prevent...,NaN,7334574748,NaN,3
4,"0.01 is the debut studio album of H3llb3nt, re...",NaN,NaN,NaN,0
...,...,...,...,...,...
923,10 Anos Depois is the tenth album by Brazilian...,NaN,NaN,NaN,0
924,10 Anti-Aircraft Regiment is an artillery regi...,NaN,9875975967,NaN,3
925,10 April 2009.The film was failed at the box o...,NaN,NaN,NaN,0
926,10 Aquarii (abbreviated 10 Aqr) is a A-type ma...,NaN,NaN,NaN,0


In [37]:
import re
patterns = {
    'email': r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})',
    'phone_number': r'(\b\d{10}\b)',
    'api_key': r'(?i)api\s*[:=]?\s*([a-zA-Z0-9-_]{8,})'
}

def calculate_risk(row):
    score = 0
    if pd.notna(row['email']): score += 1
    if pd.notna(row['phone_number']): score += 3
    if pd.notna(row['api_key']): score += 5
    return score

for col, regex in patterns.items():
    data[col] = data['Entry'].str.extract(regex)

data['risk_score'] = data.apply(calculate_risk, axis=1)   

def mask_text_content(text):
    if pd.isna(text) or not isinstance(text, str):
        return text
    # Mask Email: 
    text = re.sub(r'([a-zA-Z0-9._%+-])[a-zA-Z0-9._%+-]*@([a-zA-Z0-9.-]+\.[a-zA-Z]{2,})', r'\1*****@\2', text)
    # Mask Phone: 
    text = re.sub(r'\b(\d)\d{7}(\d{2})\b', r'\1*******\2', text)
    # Mask API Key:
    text = re.sub(r'(?i)(api\s*[:=]?\s*[a-zA-Z0-9]{4})[a-zA-Z0-9_-]+', r'\1****************', text)
    return text
    
data['Entry'] = data['Entry'].apply(mask_text_content)

In [ ]:
data

In [39]:
data.to_excel('New_masking_datas.xlsx', index=False)